# How to Render Design into Ansys

## 1. Perform the necessary imports and create a design in Metal first.

In [ ]:
%load_ext autoreload
%autoreload 2

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [ ]:
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer

QAnsysRenderer.default_options

In [ ]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.components.qubits.transmon_pocket import TransmonPocket

In [ ]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs.

In [ ]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        a = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

from qiskit_metal.components.interconnects.meandered import RouteMeander
RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'c', 'Q2', 'b', '6.0 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'b', 'Q2', 'c', '6.1 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'c', 'Q4', 'b', '6.0 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'c', '6.1 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2. Registration occurs automatically.

In [ ]:
fourq = design.renderers.ansys # Access Ansys renderer

In [ ]:
fourq.open_ansys_design # Open either a new or existing design based on default options

In [ ]:
fourq.render_design() # Start rendering

# Miscellaneous pyEPR/Ansys commands

In [ ]:
import pyEPR as epr

# Connect to Ansys directly from notebook:

pinfo = epr.ProjectInfo(project_path = None, 
                        project_name = None,
                        design_name  = None)
modeler = pinfo.design.modeler

# Get project and design names:

pinfo.project_name
design._design.GetName()

# Filter qgeometry table:

full_table = design.qgeometry.tables['poly']
mask = full_table['subtract'] == False
table = full_table[mask]

# Draw centered rectangles:

bigsquare = modeler.draw_rect_center([0, 0, 0], x_size=8, y_size=8, name='bigsquare')
topright = modeler.draw_rect_center([2, 2, 0], x_size=2, y_size=2, name='topright')

# Subtracting shapes:

modeler.subtract('bigsquare', ['topright'])

# Draw centered box:

modeler.draw_box_center([0, 0, 0], [1, 2, 3])

# Draw closed polygon:

trianglepts = [[-1, 5, 0], [1, 5, 0], [0, 7, 0]]
modeler.draw_polyline(trianglepts, closed=True)

# Draw polyline:

smallpts = [[2.85, 0, 0], [3.15, 0, 0]]
modeler.draw_polyline(smallpts, closed=False)

# Sweep one polyline with another:

modeler._sweep_along_path('Polyline8', 'Polyline7')

# TODOs

#### GEOMETRY:
1. UNITS (1hr) -> done
2. Create CPW without dogleg using marcos code and test out (1 hr)
3. Bug in fillet radius for some edges (2 hrs?) -> unsolved 
Note: cpw2 has the largest fillet radius of them all and won't even draw after shrinking down to the correct units (mm).
Worth noting that to_fillet function in hfss_renderer is basically identical to the code starting at line 284 in draw_cpw.py (v0.1) except:
(1) Lambda function dist replaced by nested non-lambda function dist for clarity.
(2) norm(arr[j] - arr[k]) instead of abs(norm(arr[j] - arr[k])) since norms are non-negative.
(3) min(dist(i, i - 1), dist(i, i + 1)) <= 2 * fillet instead of (dist(i, i - 1) <= fillet or dist(i, i + 1) <= fillet).
4. Some more test designs with images side by side of metal / hfss 
5. Refractor code into abstractions

#### Boundary conditions - perfE
0. Abstract render_path and poly into functions
1. Make perf_E everything that is not a helper!! -> get name of created hfss object and then assign a perfE BC use v0.1dev code) https://github.ibm.com/IBM-Q-Restricted-Research/qiskit-metal/blob/master/qiskit_metal/objects/qubits/Metal_Transmon_Pocket.py

MAKE ALL by default that is a poly or path (non-zero width) and NOT  a helper and 
```
            oModeler.append_PerfE_assignment(name+'_pads' if DEFAULT['BC_individual'] else options_hfss['BC_name_pads'],
                                             [hfss_objs['pad_top'],
                                              hfss_objs['pad_bot']])

            for key in self.options.connectors:
                oModeler.append_PerfE_assignment(name+'_conn' if DEFAULT['BC_individual'] else options_hfss['BC_name_conn'],
                                                 [hfss_objs.connectors[key]['connector_pad'],
                                                  hfss_objs.connectors[key]['connector_wire']])
           SEE PYEPR
```


### CHIP Substrate and Chip sample holder vacuum box 
##### Use v0.1 CODE!!!!!!!!

https://github.ibm.com/IBM-Q-Restricted-Research/qiskit-metal/blob/46d113e59b92936c7bfaf57b37e677bfbebca893/qiskit_metal/draw_functions.py#L78
DONT use region. Just draw a vacuum box

        self._chips['main']['size'] = Dict(
            center_x='0.0mm',
            center_y='0.0mm',
            center_z='0.0mm',
            size_x='9mm',
            size_y='6mm',
            size_z='-750um',
            sample_holder_top='890um', # how tall is the vacuum above z=0
            sample_holder_bottom='1650um' # how tall is the vacuum below z=0
        )
  
* render_chip
https://github.ibm.com/IBM-Q-Restricted-Research/qiskit-metal/blob/46d113e59b92936c7bfaf57b37e677bfbebca893/qiskit_metal/draw_functions.py#L136
* render_sample_holder



**SOON: Junction table (Priti & Thomas)**

A new tabl for junctions -> two points -> draw a rectangle with these lumeped BC -> Ask Thomas 

            rect_jj = hfss_objs['rect_jj'] # pylint: disable=invalid-name # pyEPR.hfss.Rect
            axis= self.options.orientation
            axis = {90:'x',-90:'x',0:'y',180:'y'}.get(axis,axis)
            axis = axis.lower()
            rect_jj.make_rlc_boundary(axis, l=options_hfss['Lj'],
                                      c=options_hfss['Cj'], r=options_hfss['_Rj'],
                                      name='Lj_'+name)

            [start, end] = rect_jj.make_center_line(axis)

            poly_jj = oModeler.draw_polyline([start, end], closed=False,
                                             **{**rect_options, **dict(color=(128, 0, 128))})
            poly_jj = poly_jj.rename('JJ_'+name+'_')
            poly_jj.show_direction = True
            
MESH 
  # JJ
            design.mesh_obj(
                hfss_objs['rect_jj'], options_hfss['mesh_name_jj'], **options_hfss['mesh_kw_jj'])